In [ ]:
from czbenchmarks.datasets.utils import load_dataset
from czbenchmarks.runner import run_inference
from czbenchmarks.tasks import (
    PerturbationTask,
)
import time
import json

In [ ]:
dataset = load_dataset("norman_perturb")
dataset.load_data()

AttributeError: 'AnnData' object has no attribute 'split'

In [29]:
dataset = run_inference("SCGENEPT", dataset=dataset, gene_pert="ETS2+MAPK1",dataset_name="norman",model_variant="scgenept_ncbi+uniprot_gpt")

ReadTimeout: UnixHTTPConnectionPool(host='localhost', port=None): Read timed out. (read timeout=60)

In [ ]:
task = PerturbationTask()
perturbation_results = task.run(dataset)

print(perturbation_results)

In [ ]:
dataset.adata.obs.split.unique()

['train']
Categories (1, object): ['train']

In [28]:
dataset.adata.obs.condition_name.unique()

['A549_ctrl_1']
Categories (1, object): ['A549_ctrl_1']

In [ ]:
species = ["human","mouse","rhesus_macaque","gorilla","chimpanzee","marmoset","chicken","opossum","platypus"]

In [ ]:
from czbenchmarks.datasets.utils import load_dataset
from czbenchmarks.runner import run_inference
from czbenchmarks.tasks import CrossSpeciesIntegrationTask
model_variant = "33l"
for species in ["human","mouse","rhesus_macaque"]:
    dataset = load_dataset(f"{species}_spermatogenesis")
    dataset = run_inference("UCE", dataset=dataset, model_variant=model_variant,gpu=True)
    task = CrossSpeciesIntegrationTask(label_key="cell_type")
    clustering_results = task.run(dataset)

In [5]:
model_variant = "33l"
dataset_name = "human"
dataset = load_dataset(f"{dataset_name}_spermatogenesis")

In [7]:
dataset.organism

homo_sapiens

In [ ]:
model_variant = "33l"
dataset_name = "human"
print(dataset_name)
dataset = load_dataset(f"{dataset_name}_spermatogenesis")
start_time = time.perf_counter()
dataset = run_inference("UCE", dataset=dataset, model_variant=model_variant,gpu=True)
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"{model_variant}, {dataset_name}: {elapsed_time}\n")

human


In [5]:
condition="BHLHE40+ctrl"
dataset_name="adamson"
model_variant="scgenept_ncbi+uniprot_gpt"
dataset = load_dataset("adamson_perturb")
dataset = run_inference("SCGENEPT", dataset=dataset, gene_pert=condition,dataset_name=dataset_name,model_variant=model_variant,interactive=True,app_mount_dir="/home/ssm-user/cz-benchmarks")

APIError: 409 Client Error for http+docker://localhost/v1.45/containers/5e35a6afeb027fcc6cfc94a89472e1cb481b96b181dd1f07175edee05fe1e603?v=False&link=False&force=False: Conflict ("cannot remove container "/friendly_lewin": container is running: stop the container before removing or force remove")

In [ ]:
workflows = {   
        "adamson":
            [
            "BHLHE40+ctrl",
            "CAD+ctrl"
            ],
        "norman":
            [
            "ETS2+MAPK1",
            "POU3F2+CBFA2T3"
            ]
        }

model_variants = [
    "scgenept_ncbi+uniprot_gpt",
    "scgenept_go_c_gpt_concat",
    "scgenept_go_all_gpt_concat"
]
perf_log = list()
for dataset_name, conditions in workflows.items():
    dataset = load_dataset(dataset_name,"perf_analysis.yaml")
    for condition in conditions:
        for model_variant in model_variants:
            start_time = time.perf_counter()
            dataset = run_inference("SCGENEPT", dataset=dataset, gene_pert=condition,dataset_name=dataset_name,model_variant=model_variant)
            end_time = time.perf_counter()
            elapsed_time = end_time - start_time
            print(f"{model_variant}, {dataset_name}, {condition}: {elapsed_time}\n")
            perf_log.append({"model_variant": model_variant,"dataset":dataset_name,"condition":condition,"runtime":elapsed_time})

with open('scgenept_perf_results.json', 'w') as file:
    json.dump(perf_log, file, indent=4)